# Stacking Strong Neural Optimal Transport (SNOT)

This notebook collects notebooks from *"stacking_notebooks"* folder 
in one place.

It was implemented with manual selection of the weights for the prediction with preferable FID-metric of the previous stacking stage.

## Import libraries

In [ ]:
import os, sys
# sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src import distributions
import torch.nn.functional as F

from src.resnet2 import ResNet_D
from src.unet import UNet

from src.tools import unfreeze, freeze
from src.tools import weights_init_D
from src.tools import load_dataset, get_pushed_loader_stats
from src.fid_score import calculate_frechet_distance
from src.plotters import plot_random_images, plot_images

# Module for saving and loading data from previous stacking step
from src.transform_save_load import transform_data, new_sample_the_same

from copy import deepcopy
import json

from tqdm import tqdm
from IPython.display import clear_output

import wandb # <--- online logging of the results
from src.tools import fig2data, fig2img # for wandb

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

## Main Config

In case of uniform model parameters during stacking steps

In [ ]:
DEVICE_IDS = [0]

DATASET1 = 'handbag' # Path will be different from stage to stage

# Target (Always the same)
DATASET2  = 'shoes'
DATASET2_PATH = 'data/shoes_64.hdf5'

IMG_SIZE = 64

T_ITERS = 10
f_LR, T_LR = 1e-4, 1e-4
BATCH_SIZE = 64

PLOT_INTERVAL = 100
COST = 'mse' # Mean Squared Error
CPKT_INTERVAL = 500

SEED = 0x000000

### Preparation

In [ ]:
config = dict(
    DATASET1=DATASET1,
    DATASET2=DATASET2, 
    T_ITERS=T_ITERS,
    f_LR=f_LR, T_LR=T_LR,
    BATCH_SIZE=BATCH_SIZE
)
    
assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_IDS[0]}')
torch.manual_seed(SEED); np.random.seed(SEED)

## Loading data stats for testing
This stats data should be on the root folder "data". It can be created by [compute_stats.ipynb](https://github.com/iamalexkorotin/NeuralOptimalTransport/blob/main/stats/compute_stats.ipynb)

In [ ]:
# Always the same
filename = 'data/{}_{}_test.json'.format(DATASET2, IMG_SIZE)
with open(filename, 'r') as fp:
    data_stats = json.load(fp)
    mu_data, sigma_data = data_stats['mu'], data_stats['sigma']
del data_stats

## Prepare Samplers Y
They will not change during stacking

In [ ]:
# Always the same
Y_sampler, Y_test_sampler = load_dataset(DATASET2, DATASET2_PATH, img_size=IMG_SIZE)

## Training function

Same commands for all stacking steps, 
thus for making this notebook more compact these commands were combined 
in one function

In [ ]:
def training():
    """Training function for all stacking steps.
    Initialize MAX_STEPS, OUTPUT_PATH, T, f, T_opt, f_opt, 
    X_sampler, X_fixed, Y_fixed, X_test_fixed, Y_test_fixed, X_test_sampler, 
    Y_test_sampler before launching this function!
    """
    for step in tqdm(range(MAX_STEPS)):
        # T optimization
        unfreeze(T); freeze(f)
        for t_iter in range(T_ITERS): 
            T_opt.zero_grad()
            X = X_sampler.sample(BATCH_SIZE)
            T_X = T(X)
            if COST == 'mse':
                T_loss = F.mse_loss(X, T_X).mean() - f(T_X).mean()
            else:
                raise Exception('Unknown COST')
            T_loss.backward(); T_opt.step()
        del T_loss, T_X, X; gc.collect(); torch.cuda.empty_cache()

        # f optimization
        freeze(T); unfreeze(f)
        X = X_sampler.sample(BATCH_SIZE)
        with torch.no_grad():
            T_X = T(X)
        Y = Y_sampler.sample(BATCH_SIZE)
        f_opt.zero_grad()
        f_loss = f(T_X).mean() - f(Y).mean()
        f_loss.backward(); f_opt.step();
        wandb.log({f'f_loss' : f_loss.item()}, step=step) 
        del f_loss, Y, X, T_X; gc.collect(); torch.cuda.empty_cache()
            
        if step % PLOT_INTERVAL == 0:
            print('Plotting')
            clear_output(wait=True)
            
            fig, axes = plot_images(X_fixed, Y_fixed, T)
            wandb.log({'Fixed Images' : [wandb.Image(fig2img(fig))]}, 
                      step=step) 
            plt.show(fig); plt.close(fig) 
            
            fig, axes = plot_random_images(X_sampler,  Y_sampler, T)
            wandb.log({'Random Images' : [wandb.Image(fig2img(fig))]}, 
                      step=step) 
            plt.show(fig); plt.close(fig) 
            
            fig, axes = plot_images(X_test_fixed, Y_test_fixed, T)
            wandb.log({'Fixed Test Images' : [wandb.Image(fig2img(fig))]}, 
                      step=step) 
            plt.show(fig); plt.close(fig) 
            
            fig, axes = plot_random_images(X_test_sampler, Y_test_sampler, T)
            wandb.log({'Random Test Images' : [wandb.Image(fig2img(fig))]}, 
                      step=step) 
            plt.show(fig); plt.close(fig) 
        
        if step % CPKT_INTERVAL == CPKT_INTERVAL - 1:
            freeze(T); 
            
            print('Computing FID')
            mu, sigma = get_pushed_loader_stats(T, X_test_sampler.loader)
            fid = calculate_frechet_distance(mu_data, sigma_data, mu, sigma)
            wandb.log({f'FID (Test)' : fid}, step=step)
            del mu, sigma
            
            torch.save(T.state_dict(), 
                       os.path.join(OUTPUT_PATH, f'{SEED}_{step}.pt'))
            torch.save(f.state_dict(), 
                       os.path.join(OUTPUT_PATH, f'f_{SEED}_{step}.pt'))
            torch.save(f_opt.state_dict(), 
                       os.path.join(OUTPUT_PATH, f'f_opt_{SEED}_{step}.pt'))
            torch.save(T_opt.state_dict(), 
                       os.path.join(OUTPUT_PATH, f'T_opt_{SEED}_{step}.pt'))
        
        gc.collect(); torch.cuda.empty_cache()

# Stacking #0

This is a modified baseline of [NOT_strong](https://github.com/iamalexkorotin/NeuralOptimalTransport/blob/main/notebooks/NOT_training_strong.ipynb)

## Stacking Config

In [ ]:
STAGE = 0

DATASET1_SOURCE_PATH = 'data/handbag_64.hdf5'

MAX_STEPS = 1001 # for testing one-notebook solution, default = 100_001

EXP_NAME = f'_{DATASET1}_{DATASET2}_T{T_ITERS}_{COST}_{IMG_SIZE}_S{STAGE}'
OUTPUT_PATH = f'checkpoints/{COST}/{DATASET1}_{DATASET2}_{IMG_SIZE}/stage_{STAGE}'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

## Prepare Sampler X

In [ ]:
X_sampler, X_test_sampler = load_dataset(DATASET1, DATASET1_SOURCE_PATH, 
                                         img_size=IMG_SIZE)
    
torch.cuda.empty_cache(); gc.collect()
# clear_output()

## Initializing Networks

In [ ]:
f = ResNet_D(IMG_SIZE, nc=3).cuda()
f.apply(weights_init_D)

T = UNet(3, 3, base_factor=48).cuda()

if len(DEVICE_IDS) > 1:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    f = nn.DataParallel(f, device_ids=DEVICE_IDS)
    
print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

In [ ]:
torch.manual_seed(0xBADBEEF); np.random.seed(0xBADBEEF)
X_fixed = X_sampler.sample(10)
Y_fixed = Y_sampler.sample(10)
X_test_fixed = X_test_sampler.sample(10)
Y_test_fixed = Y_test_sampler.sample(10)

### Plots Test

In [ ]:
fig, axes = plot_images(X_fixed, Y_fixed, T)
fig, axes = plot_random_images(X_sampler, Y_sampler, T)
fig, axes = plot_images(X_test_fixed, Y_test_fixed, T)
fig, axes = plot_random_images(X_test_sampler, Y_test_sampler, T)

## Run Training

In [ ]:
wandb.init(name=EXP_NAME, project='snot', entity='dstech', config=config)
pass

In [ ]:
T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=f_LR, weight_decay=1e-10)

training()

## Finish experiment

In [ ]:
# Finish experiment
gc.collect(); torch.cuda.empty_cache()
wandb.finish()

# Stacking #1

## Make prediction

According to obtained FID-metric choose preferable step (for example, with minimal FID-metric) and save weights *"0_[preferable step].pt*" from checkpoints as *"best_weights/stage_[STAGE].pt"*

## Stacking Config

In [ ]:
STAGE = 1

# Only for stage 1, before getting the prediction
DATASET1_SOURCE_PATH = 'data/handbag_64.hdf5'

# Path for getting the best weights from last stage
BEST_WEIGHTS_PATH = f'best_weights/stage_{STAGE-1}.pt'

# Path for saving prediction images by the previous stage
# and for loading from them for the current stage
DATASET1_TRAIN_PATH = f'data/handbag_64_train_stage_{STAGE}.pt'
DATASET1_TEST_PATH = f'data/handbag_64_test_stage_{STAGE}.pt'

MAX_STEPS = 501 # for testing one-notebook solution, default = 30_001

EXP_NAME = f'_{DATASET1}_{DATASET2}_T{T_ITERS}_{COST}_{IMG_SIZE}_S{STAGE}'
OUTPUT_PATH = f'checkpoints/{COST}/{DATASET1}_{DATASET2}_{IMG_SIZE}/stage_{STAGE}'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

## Prepare Previous Sampler X, if it is not on the RAM

In [ ]:
# # Only for the stage 1
# X_sampler, X_test_sampler = load_dataset(DATASET1, 
#                             DATASET1_SOURCE_PATH, img_size=IMG_SIZE)

# torch.cuda.empty_cache(); gc.collect()

## Set up T(X) on best weights

In [ ]:
T = UNet(3, 3, base_factor=48)

print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))

# Setup earlier params
T.load_state_dict(torch.load(BEST_WEIGHTS_PATH))

## Predict on best T(X) for train and test, and save results

In [ ]:
transform_data(X_sampler, DATASET1_TRAIN_PATH, T)
del X_sampler

In [ ]:
transform_data(X_test_sampler, DATASET1_TEST_PATH, T)
del X_test_sampler

## Prepare New Samplers X

In [ ]:
X_sampler = new_sample_the_same(DATASET1_TRAIN_PATH, shuffle=True)
X_test_sampler = new_sample_the_same(DATASET1_TEST_PATH)

torch.cuda.empty_cache(); gc.collect()

## Initializing Networks

In [ ]:
f = ResNet_D(IMG_SIZE, nc=3).cuda()
f.apply(weights_init_D)

T = UNet(3, 3, base_factor=48).cuda()

if len(DEVICE_IDS) > 1:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    f = nn.DataParallel(f, device_ids=DEVICE_IDS)
    
print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

In [ ]:
torch.manual_seed(0xBADBEEF); np.random.seed(0xBADBEEF)
X_fixed = X_sampler.sample(10)
Y_fixed = Y_sampler.sample(10)
X_test_fixed = X_test_sampler.sample(10)
Y_test_fixed = Y_test_sampler.sample(10)

### Plots Test

In [ ]:
fig, axes = plot_images(X_fixed, Y_fixed, T)
fig, axes = plot_random_images(X_sampler, Y_sampler, T)
fig, axes = plot_images(X_test_fixed, Y_test_fixed, T)
fig, axes = plot_random_images(X_test_sampler, Y_test_sampler, T)

## Run Training

In [ ]:
wandb.init(name=EXP_NAME, project='snot', entity='dstech', config=config)
pass

In [ ]:
T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=f_LR, weight_decay=1e-10)

training()

## Finish experiment

In [ ]:
# Finish experiment
gc.collect(); torch.cuda.empty_cache()
wandb.finish()

# Stacking #2
This Stacking-stage can be duplicated to continue stacking, if it is necessary

## Make prediction

According to obtained FID-metric choose preferable step (for example, with minimal FID-metric) and save weights *"0_[preferable step].pt*" from checkpoints as *"best_weights/stage_[STAGE].pt"*

## Stacking Config

In [ ]:
STAGE = 2

# Only for stage 1, before getting the prediction
# DATASET1_SOURCE_PATH = 'data/handbag_64.hdf5'

# For the next stages (STAGE > 1)
DATASET1_SOURCE_TRAIN_PATH =  f'data/handbag_64_train_stage_{STAGE-1}.pt'
DATASET1_SOURCE_TEST_PATH =  f'data/handbag_64_test_stage_{STAGE-1}.pt'

# Path for getting the best weights from last stage
BEST_WEIGHTS_PATH = f'best_weights/stage_{STAGE-1}.pt'

# Path for saving prediction images by the previous stage
# and for loading from them for the current stage
DATASET1_TRAIN_PATH = f'data/handbag_64_train_stage_{STAGE}.pt'
DATASET1_TEST_PATH = f'data/handbag_64_test_stage_{STAGE}.pt'

MAX_STEPS = 501 # for testing one-notebook solution, default = 30_001

EXP_NAME = f'_{DATASET1}_{DATASET2}_T{T_ITERS}_{COST}_{IMG_SIZE}_S{STAGE}'
OUTPUT_PATH = f'checkpoints/{COST}/{DATASET1}_{DATASET2}_{IMG_SIZE}/stage_{STAGE}'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

## Prepare Previous Sampler X, if it is not on the RAM

In [ ]:
# # For the stage 2 and next
# X_sampler = new_sample_the_same(DATASET1_SOURCE_TRAIN_PATH)
# X_test_sampler = new_sample_the_same(DATASET1_SOURCE_TEST_PATH)
    
# torch.cuda.empty_cache(); gc.collect()
# # clear_output()

## Set up T(X) on best weights

In [ ]:
T = UNet(3, 3, base_factor=48)

print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))

# Setup earlier params
T.load_state_dict(torch.load(BEST_WEIGHTS_PATH))

## Predict on best T(X) for train and test, and save results

In [ ]:
transform_data(X_sampler, DATASET1_TRAIN_PATH, T)
del X_sampler

In [ ]:
transform_data(X_test_sampler, DATASET1_TEST_PATH, T)
del X_test_sampler

## Prepare New Samplers X

In [ ]:
X_sampler = new_sample_the_same(DATASET1_TRAIN_PATH, shuffle=True)
X_test_sampler = new_sample_the_same(DATASET1_TEST_PATH)

torch.cuda.empty_cache(); gc.collect()

## Initializing Networks

In [ ]:
f = ResNet_D(IMG_SIZE, nc=3).cuda()
f.apply(weights_init_D)

T = UNet(3, 3, base_factor=48).cuda()

if len(DEVICE_IDS) > 1:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    f = nn.DataParallel(f, device_ids=DEVICE_IDS)
    
print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

In [ ]:
torch.manual_seed(0xBADBEEF); np.random.seed(0xBADBEEF)
X_fixed = X_sampler.sample(10)
Y_fixed = Y_sampler.sample(10)
X_test_fixed = X_test_sampler.sample(10)
Y_test_fixed = Y_test_sampler.sample(10)

### Plots Test

In [ ]:
fig, axes = plot_images(X_fixed, Y_fixed, T)
fig, axes = plot_random_images(X_sampler, Y_sampler, T)
fig, axes = plot_images(X_test_fixed, Y_test_fixed, T)
fig, axes = plot_random_images(X_test_sampler, Y_test_sampler, T)

## Run Training

In [ ]:
wandb.init(name=EXP_NAME, project='snot', entity='dstech', config=config)
pass

In [ ]:
T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=f_LR, weight_decay=1e-10)

training()

## Finish experiment

In [ ]:
# Finish experiment
gc.collect(); torch.cuda.empty_cache()
wandb.finish()

So 3 Stacking-like experiments were carried out. See the results in wandb (where you logged them)